In [5]:
import tensorflow.keras.applications as models

model = models.ResNet50(weights=None)
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [2]:
from tensorflow import keras
from tensorflow.keras.applications import VGG16

def get_pretrained_model(input_shape, n_classes, freezed=True):
    base_model = VGG16(input_shape=input_shape,
                       weights='imagenet', include_top=False)
    base_model.trainable = not freezed

    inputs = keras.Input(shape=input_shape)
    x = base_model(inputs)

    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(256)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    outputs = keras.layers.Dense(n_classes)(x)
    return keras.Model(inputs, outputs)

model = get_pretrained_model(input_shape=(224, 224, 3), 
                             n_classes=37, freezed=True)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 re_lu_1 (ReLU)              (None, 256)               0   

In [ ]:
## Feature Extraction
model = get_pretrained_model(input_shape=(224, 224, 3), n_classes=37, freezed=True)
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics={'acc': accuracy}

clf = TrainerWithEarlyStopping(model, optimizer, loss_fn, metrics)
hist = clf.fit(train_loader, n_epochs=100, valid_loader=valid_loader)

## Fine Tuneing
model.trainable = True

optimizer = keras.optimizers.Adam(learning_rate=1e-5)

clf = TrainerWithEarlyStopping(model, optimizer, loss_fn, metrics={'acc': accuracy})
hist = clf.fit(train_loader, n_epochs=100, valid_loader=valid_loader)

In [ ]:
import segmentation_models as sm

model = sm.Unet('vgg16', input_shape=(224, 224, 3), classes=3, encoder_freeze=True)
optimizer = keras.optimizers.Adam()
loss_fn = sm.losses.DiceLoss()
metrics = {'iou': sm.metrics.IOUScore(), 'f': sm.metrics.FScore(), 'acc': acc}

clf = TrainerWithEarlyStopping(model, optimizer, loss_fn, metrics)
hist = clf.fit(train_loader, n_epochs=5, valid_loader=valid_loader)